In [79]:
import sys
import os
import numpy as np
import pandas as pd
import yfinance as yf
import scipy.optimize as scpopt
from tqdm import tqdm
from bcb import sgs
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import google.generativeai as genai
from datetime import datetime
from openai import OpenAI
import plotly

In [80]:
#pip install google.generativeai

In [81]:
#pip install plotly

Períodos do modelo

In [82]:
data_inicio = '2012-01-01'
data_fim = '2025-07-01'
data_backtest = "2022-07-01" #2022 foi o melhor 

Obtenção das variáveis macroeconômicas

In [83]:
codigos_sgs = {
    'SELIC': 4189, # Usando a SELIC mensalizada diretamente
    'IPCA': 433,
    'IBC-Br': 24369,
    'DESEMPREGO': 24369,
    'Investimento Direto no País': 22886,
}

dados_bcb = pd.DataFrame()

# Buscar e juntar cada série
for nome_serie, codigo in codigos_sgs.items():
    try:
        print(f"Buscando: {nome_serie} (código {codigo})...")
        serie = sgs.get({nome_serie: codigo}, start= data_inicio, end= data_fim)

        # Se o DataFrame principal ainda estiver vazio, ele se torna a primeira série
        if dados_bcb.empty:
            dados_bcb = serie
        # Caso contrário, juntamos (join) a nova série ao DataFrame existente
        else:
            dados_bcb = dados_bcb.join(serie)

    except Exception as e:
        print(f"AVISO: Falha ao buscar a série '{nome_serie}' (código {codigo}). Erro: {e}. Pulando...")

# 3. Verificação final
if not dados_bcb.empty:
    print("\nDados do BCB e IBGE obtidos com sucesso (método join):")
    display(dados_bcb.tail())
else:
    print("\nNenhuma série do Banco Central pôde ser baixada.")

# Lista de dados de mercado
tickers_mercado = {
    'VIX': '^VIX',
    'PETROLEO_BRENT': 'BZ=F',
    'DOW_JONES': '^DJI',
    #'IMAT': 'IMAT.SA', Não achou
    'COMM_AGRICOLA': 'DBA', # Usando o ETF 'DBA'
    'CAMBIO_USDBRL': 'USDBRL=X'
}

print("\nBuscando dados de índices de mercado e commodities...")
# Mesma lógica de baixar dado diário para mensal
dados_mercado_diario_raw = yf.download(
    list(tickers_mercado.values()),
    start= data_inicio, end= data_fim,
    auto_adjust=True
)['Close']

mapa_renomear = {v: k for k, v in tickers_mercado.items()}

# Usamos o método .rename() que é explícito e não depende da ordem
dados_mercado_diario = dados_mercado_diario_raw.rename(columns=mapa_renomear)

# Reordenar as colunas para a ordem certa
dados_mercado_diario = dados_mercado_diario[list(tickers_mercado.keys())]

print("\nDados de Mercado (diários) - Corrigidos e Verificados:")
display(dados_mercado_diario.tail())

# 1. Converter dados de mercado para o final do mês
dados_mercado_mensal = dados_mercado_diario.resample('ME').last()

print("Índice original do 'dados_bcb':")
display(dados_bcb.head(2))

dados_bcb.index = dados_bcb.index.to_period('M').to_timestamp('M')

print("\nÍndice padronizado do 'dados_bcb' (agora com fim de mês):")
display(dados_bcb.head(2))


#Juntando os dados
dados_macro_combinados = dados_bcb.join(dados_mercado_mensal)


# O ffill() ainda é útil para cobrir feriados ou dias em que um dos mercados não abriu.
# O dropna() remove qualquer linha no início que possa ter ficado sem dados completos.
dados_macro_combinados = dados_macro_combinados.ffill().dropna()

dados_macro_combinados.index = dados_macro_combinados.index.to_period('M').to_timestamp('M')
dados_macro_combinados.index = pd.to_datetime(dados_macro_combinados.index)

print("\n--- DataFrame Final com Todos os Indicadores Macroeconômicos ---")
display(dados_macro_combinados.tail())
display(dados_macro_combinados.head())

Buscando: SELIC (código 4189)...
Buscando: IPCA (código 433)...
Buscando: IBC-Br (código 24369)...
Buscando: DESEMPREGO (código 24369)...
Buscando: Investimento Direto no País (código 22886)...

Dados do BCB e IBGE obtidos com sucesso (método join):


,SELIC,IPCA,IBC-Br,DESEMPREGO,Investimento Direto no País
Date,,,,,
2025-03-01,13.57,0.56,7.0,7.0,11331.8
2025-04-01,14.15,0.43,6.6,6.6,14267.0
2025-05-01,14.55,0.26,6.2,6.2,11125.6
2025-06-01,14.74,0.24,5.8,5.8,12496.3
2025-07-01,14.90,0.26,5.6,5.6,17303.5


[*********************100%***********************]  5 of 5 completed


Buscando dados de índices de mercado e commodities...

Dados de Mercado (diários) - Corrigidos e Verificados:


Ticker,VIX,PETROLEO_BRENT,DOW_JONES,COMM_AGRICOLA,CAMBIO_USDBRL
Date,,,,,
2025-06-24,17.48,67.139999,43089.019531,26.290001,5.4938
2025-06-25,16.76,67.680000,42982.429688,26.080000,5.5103
2025-06-26,16.59,67.730003,43386.839844,26.139999,5.5579
2025-06-27,16.32,67.769997,43819.269531,26.330000,5.4771
2025-06-30,16.73,67.610001,44094.769531,26.150000,5.4784


...


Índice padronizado do 'dados_bcb' (agora com fim de mês):


,SELIC,IPCA,IBC-Br,DESEMPREGO,Investimento Direto no País
Date,,,,,
2012-01-31,10.7,0.56,NaN,NaN,10670.8
2012-02-29,10.4,0.45,NaN,NaN,6216.9



--- DataFrame Final com Todos os Indicadores Macroeconômicos ---


,SELIC,IPCA,IBC-Br,DESEMPREGO,Investimento Direto no País,VIX,PETROLEO_BRENT,DOW_JONES,COMM_AGRICOLA,CAMBIO_USDBRL
Date,,,,,,,,,,
2025-03-31,13.57,0.56,7.0,7.0,11331.8,22.280001,74.739998,42001.761719,26.340000,5.7581
2025-04-30,14.15,0.43,6.6,6.6,14267.0,24.700001,63.119999,40669.359375,26.809999,5.6200
2025-05-31,14.55,0.26,6.2,6.2,11125.6,18.570000,63.900002,42270.070312,26.700001,5.6651
2025-06-30,14.74,0.24,5.8,5.8,12496.3,16.730000,67.610001,44094.769531,26.150000,5.4784
2025-07-31,14.90,0.26,5.6,5.6,17303.5,16.730000,67.610001,44094.769531,26.150000,5.4784


,SELIC,IPCA,IBC-Br,DESEMPREGO,Investimento Direto no País,VIX,PETROLEO_BRENT,DOW_JONES,COMM_AGRICOLA,CAMBIO_USDBRL
Date,,,,,,,,,,
2012-03-31,9.82,0.21,8.0,8.0,16549.8,15.500000,122.879997,13212.040039,25.025694,1.8208
2012-04-30,9.35,0.64,7.8,7.8,6664.0,17.150000,119.470001,13213.629883,24.304314,1.8860
2012-05-31,8.87,0.36,7.7,7.7,6263.8,24.059999,101.870003,12393.450195,23.226694,2.0149
2012-06-30,8.39,0.08,7.6,7.6,9004.9,17.080000,97.800003,12880.089844,25.132565,2.0768
2012-07-31,8.07,0.43,7.5,7.5,10912.8,18.930000,104.919998,13008.679688,26.833599,2.0403


In [84]:
# Criando um novo DataFrame para as features aprimoradas
dados_macro_features = pd.DataFrame(index=dados_macro_combinados.index)

# Calculando a variação mensal (a diferença) para cada indicador
for coluna in dados_macro_combinados.columns:
    dados_macro_features[f'{coluna}_diff'] = dados_macro_combinados[coluna].pct_change()

# Removendo a primeira linha que terá valores NaN (vazios)
dados_macro_features = dados_macro_features.dropna()

print("Comparando os dados originais com as novas features (variações):")
display(dados_macro_combinados.head(3))
display(dados_macro_combinados.tail(3))
display(dados_macro_features.tail(3))

dados_macro_combinados = dados_macro_features

Comparando os dados originais com as novas features (variações):


,SELIC,IPCA,IBC-Br,DESEMPREGO,Investimento Direto no País,VIX,PETROLEO_BRENT,DOW_JONES,COMM_AGRICOLA,CAMBIO_USDBRL
Date,,,,,,,,,,
2012-03-31,9.82,0.21,8.0,8.0,16549.8,15.500000,122.879997,13212.040039,25.025694,1.8208
2012-04-30,9.35,0.64,7.8,7.8,6664.0,17.150000,119.470001,13213.629883,24.304314,1.8860
2012-05-31,8.87,0.36,7.7,7.7,6263.8,24.059999,101.870003,12393.450195,23.226694,2.0149


,SELIC,IPCA,IBC-Br,DESEMPREGO,Investimento Direto no País,VIX,PETROLEO_BRENT,DOW_JONES,COMM_AGRICOLA,CAMBIO_USDBRL
Date,,,,,,,,,,
2025-05-31,14.55,0.26,6.2,6.2,11125.6,18.57,63.900002,42270.070312,26.700001,5.6651
2025-06-30,14.74,0.24,5.8,5.8,12496.3,16.73,67.610001,44094.769531,26.150000,5.4784
2025-07-31,14.90,0.26,5.6,5.6,17303.5,16.73,67.610001,44094.769531,26.150000,5.4784


,SELIC_diff,IPCA_diff,IBC-Br_diff,DESEMPREGO_diff,Investimento Direto no País_diff,VIX_diff,PETROLEO_BRENT_diff,DOW_JONES_diff,COMM_AGRICOLA_diff,CAMBIO_USDBRL_diff
Date,,,,,,,,,,
2025-05-31,0.028269,-0.395349,-0.060606,-0.060606,-0.220186,-0.248178,0.012357,0.039359,-0.004103,0.008025
2025-06-30,0.013058,-0.076923,-0.064516,-0.064516,0.123202,-0.099085,0.058059,0.043168,-0.020599,-0.032956
2025-07-31,0.010855,0.083333,-0.034483,-0.034483,0.384690,0.000000,0.000000,0.000000,0.000000,0.000000


taxas de juros americanas e o índice de ações internacional (MSCI) 

VIX e retornos do Dow Jones





In [85]:
ativos = ['VALE3.SA', 'PETR4.SA', 'ITUB4.SA', 'MRFG3.SA', 'ABEV3.SA', 'WEGE3.SA',
            'EGIE3.SA', 'LREN3.SA', 'BBAS3.SA', 'SLCE3.SA' ] #'BOVA11.SA', 'IVVB11.SA', 'IMAB11.SA', 'GLD']

#SLCE3, TTEN3, SOJA3

# IMAB11 só tem dados no yfinance a partir de maio de 2019, 'GLD' é ETF de ouro americano, o ETF br só tem dados a partir de 2021

n_ativos   = len(ativos)
carteira = sorted(ativos)

In [86]:
# 1. Baixar dados base (Ibovespa e Selic)
ibov_dados = yf.download('^BVSP', start=data_inicio, end=data_fim, interval="1mo")['Close']
ibov_retornos_brutos = np.log(ibov_dados / ibov_dados.shift(1)).dropna()

selic_mensal = sgs.get({'Selic': 4390}, start=data_inicio, end=data_fim)
#selic_mensal = (selic_anual_pct / 100) / 12

selic_mensal = selic_mensal / 100
selic_mensal.rename(columns={'Selic': 'Taxa_Livre_Risco'}, inplace=True)
print(selic_mensal)

# 2. Padronizar índices para garantir alinhamento
ibov_retornos_brutos.index = pd.to_datetime(ibov_retornos_brutos.index).to_period('M').to_timestamp('M')
selic_mensal.index = pd.to_datetime(selic_mensal.index).to_period('M').to_timestamp('M')

# 3. --- CORREÇÃO --- Calcular as métricas a partir dos RETORNOS EXCEDENTES

# Para o Ibovespa
ibov_retornos_excedentes = ibov_retornos_brutos.subtract(selic_mensal['Taxa_Livre_Risco'], axis=0).dropna()
# ibov_retorno_excedente_medio = ibov_retornos_excedentes.mean()
# ibov_volatilidade = ibov_retornos_excedentes.std() # Volatilidade do prêmio de risco
# ibov_sharpe_correto = ibov_retorno_excedente_medio / ibov_volatilidade

print(ibov_retornos_excedentes)

/tmp/ipykernel_9079/4152876746.py:2: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


            Taxa_Livre_Risco
Date                        
2012-01-01            0.0089
2012-02-01            0.0075
2012-03-01            0.0082
2012-04-01            0.0071
2012-05-01            0.0074
...                      ...
2025-03-01            0.0096
2025-04-01            0.0106
2025-05-01            0.0114
2025-06-01            0.0110
2025-07-01            0.0128

[163 rows x 1 columns]
Ticker         ^BVSP
Date                
2012-02-29  0.035025
2012-03-31 -0.028166
2012-04-30 -0.049709
2012-05-31 -0.133610
2012-06-30 -0.008881
...              ...
2025-02-28 -0.036704
2025-03-31  0.049384
2025-04-30  0.025638
2025-05-31  0.003007
2025-06-30  0.002252

[161 rows x 1 columns]


In [87]:
def retornos_mensais(data_inicio, data_fim):

    # Baixar dados diários com ajuste de dividendos
    dados = yf.download( ativos, start= data_inicio, end=data_fim, interval="1mo", auto_adjust=True)["Close"]
    #dadosF = yf.download( ativos, start="2019-06-01", end="2024-06-01", interval="1mo", auto_adjust=False)["Close"]


    dados.columns = [ticker.replace('.SA', '') for ticker in dados.columns]
    #dadosF.columns = [ticker.replace('.SA', '') for ticker in dados.columns]


    # Remover colunas totalmente vazias
    dados = dados.dropna(axis=1, how='all')
    #dadosF = dados.dropna(axis=1, how='all')

    # Calcular retorno logarítmico mensal
    retornos_absolutos = np.log(dados / dados.shift(1)).dropna()
    #retornos_mensaisF = np.log(dadosF / dadosF.shift(1)).dropna()


    # Usamos o código 4189 do SGS, que é a Selic mensalizada
    selic_anual_pct = sgs.get({'Selic': 4189}, start=data_inicio, end=data_fim)

    # Converte a taxa de % anual para decimal mensal
    selic_mensal = (selic_anual_pct / 100) / 12
    print(selic_mensal)
    selic_mensal.rename(columns={'Selic': 'Taxa_Livre_Risco'}, inplace=True)
    print("Selic baixada e convertida para taxa mensal.")


    # --- PASSO 3: GARANTIR O ALINHAMENTO DAS DATAS ---
    # Padroniza ambos os índices para o fim do mês para garantir a subtração correta
    retornos_absolutos.index = pd.to_datetime(retornos_absolutos.index).to_period('M').to_timestamp('M')
    selic_mensal.index = pd.to_datetime(selic_mensal.index).to_period('M').to_timestamp('M')


    # --- PASSO 4: CALCULAR OS RETORNOS EXCEDENTES ---
    print("Calculando os retornos excedentes (descontando a taxa livre de risco)...")
    # O método .subtract com axis=0 subtrai a série da Selic de cada coluna do DataFrame de retornos
    retornos_excedentes_mensais = retornos_absolutos.subtract(selic_mensal['Taxa_Livre_Risco'], axis=0)
    retornos_excedentes_mensais.dropna(inplace=True) # Remove possíveis NaNs da junção

    print("\n--- Amostra dos Retornos Excedentes Calculados ---")
    display(retornos_excedentes_mensais.head())
    

    return retornos_excedentes_mensais
    #print("Retornos mensais false:")
    #display(retornos_mensaisF.tail())

    # Calcular a volatilidade (desvio padrão) mensal de cada ativo
def volatilidade_mensal (retornos_mensais):
    #print("Volatilidade (Risco Mensal) de cada ativo:")
    #display(volatilidade_mensal)
    return retornos_mensais.std()


    # Calcular a matriz de covariância mensal dos retornos dos ativos
def matriz_covarianca_mensal(retornos_mensais):
    return retornos_mensais.cov()

    #print("\nMatriz de Covariância Mensal:")
    #display(matriz_covarianca_mensal)

In [88]:
def capitalizacao():

    capitalizacoes = {}

    print("Buscando a taxa de câmbio USD/BRL...")
    try:
        taxa_usd_brl = yf.Ticker('USDBRL=X').history(period='1d')['Close'].iloc[-1]
        print(f"Taxa de câmbio atual: R$ {taxa_usd_brl:.2f}")
    except Exception as e:
        print(f"Não foi possível buscar a taxa de câmbio. Usando um valor padrão. Erro: {e}")
        taxa_usd_brl = 5.44

    print("\nBuscando dados de capitalização/AUM para cada ativo...")

    for ativo in tqdm(ativos, desc="Processando Ativos"):
        cap = None
        try:
            ticker = yf.Ticker(ativo)
            info = ticker.info
            cap = info.get('marketCap', info.get('totalAssets'))

            # Se 'cap' foi encontrado, verificar moeda
            if cap is not None and info.get('currency') == 'USD':
                cap = cap * taxa_usd_brl
                
        except Exception as e:
            print(f"INFO: Não foi possível processar {ativo} via API. Verificando fallback. Erro: {e}")
        
        nome_ativo = ativo.replace('.SA', '')
        capitalizacoes[nome_ativo] = cap

    # Criar um Pandas Series com os resultados
    capitalizacoes_series = pd.Series(capitalizacoes, name="Capitalizacao_BRL").sort_index()


    # Alinhar a ordem dos ativos com a matriz de covariância
    ordem_retornos = sorted([c.replace('.SA', '') for c in ativos if c.replace('.SA', '') in capitalizacoes_series.index])
    capitalizacoes_series = capitalizacoes_series.reindex(ordem_retornos)

    print("\n--- Capitalização de Mercado / AUM (em BRL) - COMPLETO ---")
    display(capitalizacoes_series.map('{:,.0f}'.format))

    # Calcular os Pesos de Mercado (w_mkt)
    w_mkt = capitalizacoes_series / capitalizacoes_series.sum()

    print("\n--- Pesos da Carteira de Mercado (w_mkt)")
    display(w_mkt.map('{:.2%}'.format))

    # Verificação final para garantir que a soma dos pesos é 1 (ou 100%)
    print(f"\nSoma dos pesos: {w_mkt.sum():.4f}")
    
    return w_mkt 

In [89]:
# Calcular os retornos históricos do portfólio de mercado

#delta_calculado = delta(retornos_mes ,w_mkt)

#pi = (delta_calculado, w_mkt, matriz_covarianca_mes)

def delta (retornos, w_mkt):

    retornos_alinhados = retornos[w_mkt.index]
    retornos_mkt = retornos_alinhados @ w_mkt

    print("Amostra dos retornos históricos do portfólio de mercado:")
    #display(retornos_mkt.head())

    # Calcular o prêmio de risco 
    premio_risco_mkt = retornos_mkt.mean()
    print(f"\nPrêmio de Risco Mensal Médio (Numerador): {premio_risco_mkt:.4%}")

    # Calcular a variância do mercado 
    variancia_mkt = retornos_mkt.var()
    print(f"Variância Mensal do Mercado (Denominador): {variancia_mkt:.6f}")


    # Calcular o coeficiente Delta (δ)
    delta_calculado = premio_risco_mkt / variancia_mkt

    print(f"O coeficiente de aversão ao risco (δ) calculado é: {delta_calculado:.4f}")

    return delta_calculado

def pi(delta_calculado, w_mkt, matriz_cov):

    pi = delta_calculado * (matriz_cov @ w_mkt) 

    #display(pi)
    return pi

P = np.identity(len(ativos))


In [90]:
def opinioes_RF(retornos_mensais, dados_macro_historicos ):
    # Garante que ambos os DataFrames usem a mesma convenção de data (fim do mês)

    print("Padronizando os índices de data...")
    retornos_mensais.index = pd.to_datetime(retornos_mensais.index)
    retornos_mensais.index = retornos_mensais.index.to_period('M').to_timestamp('M')

    dados_macro_historicos.index = pd.to_datetime(dados_macro_historicos.index)
    dados_macro_historicos.index = dados_macro_historicos.index.to_period('M').to_timestamp('M')


    previsoes_q = {}
    ativos = retornos_mensais.columns

    for ativo in ativos:
        y = retornos_mensais[ativo]
        X = dados_macro_historicos

        X_lagged = X.shift(1)
        
        # Junta os alvos (y) e as var. com lag (X_lagged)
        dados_completos = pd.concat([y, X_lagged], axis=1)
        
        # Remove apenas as linhas que ficaram com NaN
        dados_completos.dropna(inplace=True)
        
        y_final = dados_completos[ativo]
        X_final = dados_completos[X.columns]

        if X_final.empty or y_final.empty:
            print(f" ERRO: Após o preparo, não restaram dados para o ativo {ativo}. Pulando.")
            continue

        print(f"\n--- Treinando modelo para o ativo: {ativo} ({len(X_final)} amostras) ---")

        # Treinamento do Modelo
        modelo_rf = RandomForestRegressor(
            n_estimators=200,       # Número de árvores 
            max_depth=7,            # <-- Limita a profundidade da árvore
            max_features='sqrt',    # <-- Usa um subconjunto aleatório de var. macro
            min_samples_leaf=5,     # <-- Mais amostras nas folhas finais
            random_state=42,
            oob_score=True
        )
        # ----------------------------------------------
        
        modelo_rf.fit(X_final, y_final)
        print(f"Modelo treinado. OOB Score: {modelo_rf.oob_score_:.4f}")
        
        # Geração da Previsão
        dados_macro_recentes = dados_macro_historicos.iloc[[-1]]
        previsao_retorno = modelo_rf.predict(dados_macro_recentes)
        previsoes_q[ativo] = previsao_retorno[0]


    if previsoes_q:
        q = pd.Series(previsoes_q).reindex(ativos)
        #P = np.identity(len(ativos)) P será global

        print("\n\n--- Matrizes de Visão (P e Q) Geradas ---")
        print("\nVetor Q (Previsões de Retorno Mensal %):")
        #display((q * 100).map('{:.4f}%'.format))
        #print("\nMatriz P (Visões Absolutas):")
        #display(pd.DataFrame(P, index=q.index, columns=ativos))
    else:
        print("\nNenhuma previsão pôde ser gerada pois os modelos não foram treinados.")

    return q

In [91]:
def opinioes_LR(retornos_mensais, dados_macro_historicos):
    """
    Gera o vetor de visões (Q) usando um modelo de Regressão Linear para cada ativo.
    """
    # Padronização de datas
    retornos_mensais.index = pd.to_datetime(retornos_mensais.index).to_period('M').to_timestamp('M')
    dados_macro_historicos.index = pd.to_datetime(dados_macro_historicos.index).to_period('M').to_timestamp('M')

    previsoes_q = {}
    ativos = retornos_mensais.columns

    for ativo in ativos:

        y = retornos_mensais[ativo]
        X = dados_macro_historicos
        X_lagged = X.shift(1)
        dados_completos = pd.concat([y, X_lagged], axis=1).dropna()
        y_final = dados_completos[ativo]
        X_final = dados_completos[X.columns]

        if X_final.empty or y_final.empty:
            print(f"ERRO: Não há dados suficientes para o ativo {ativo}. Pulando.")
            continue

        modelo_lr = LinearRegression() 

        # Treinamento do Modelo
        modelo_lr.fit(X_final, y_final)
        
        # Avaliação com R-quadrado (R²)
        score_r2 = modelo_lr.score(X_final, y_final)
        print(f"Modelo para {ativo} treinado. R² (ajuste de treino): {score_r2:.4f}")
        
        # Geração da Previsão
        dados_macro_recentes = dados_macro_historicos.iloc[[-1]]
        previsao_retorno = modelo_lr.predict(dados_macro_recentes)
        previsoes_q[ativo] = previsao_retorno[0]

    if not previsoes_q:
        print("\nNenhuma previsão pôde ser gerada.")
        return None

    q = pd.Series(previsoes_q).reindex(ativos)

    print("\n--- Vetor Q (Regressão Linear) Gerado ---")
    #display((q * 100).map('{:.4f}%'.format))
    
    return q

In [92]:
def opinioes_HIBRIDO(retornos_mensais, dados_macro_historicos):
    """
    Gera o vetor de visões (Q) usando um modelo HÍBRIDO:
    1. Random Forest é usado para selecionar as features mais importantes.
    2. Regressão Ridge é usada para fazer a previsão final com as features selecionadas.
    """
    # Padronização de datas
    retornos_mensais.index = pd.to_datetime(retornos_mensais.index).to_period('M').to_timestamp('M')
    dados_macro_historicos.index = pd.to_datetime(dados_macro_historicos.index).to_period('M').to_timestamp('M')

    # Hiperparâmetro: Quantas features selecionar? 
    N_FEATURES_PARA_SELECIONAR = 2

    previsoes_q = {}
    ativos = retornos_mensais.columns

    print(f"--- Gerando Visões com Modelo HÍBRIDO (RF para Seleção, Ridge para Previsão) ---")
    print(f"Selecionando as {N_FEATURES_PARA_SELECIONAR} features mais importantes para cada ativo.")

    for ativo in ativos:
        y = retornos_mensais[ativo]
        X = dados_macro_historicos
        X_lagged = X.shift(1)
        dados_completos = pd.concat([y, X_lagged], axis=1).dropna()
        y_final = dados_completos[ativo]
        X_final = dados_completos[X.columns]

        if len(X_final) < 1:
            print(f"ERRO: Não há dados suficientes para o ativo {ativo}. Pulando.")
            continue

        # RF para escolher ar variáveis macro
        
        seletor_rf = RandomForestRegressor(
            n_estimators=200, max_depth=7, max_features='sqrt',
            min_samples_leaf=5, random_state=42
        )
        seletor_rf.fit(X_final, y_final)

        # Pega a importância de cada variável e seleciona as N melhores
        importancia_features = pd.Series(seletor_rf.feature_importances_, index=X_final.columns)
        melhores_features = importancia_features.nlargest(N_FEATURES_PARA_SELECIONAR).index.tolist()
        
        print(f"\nAtivo: {ativo} | Melhores Features: {melhores_features}")

        # Regressão só com as selecionadas
        X_final_selecionado = X_final[melhores_features]
        dados_macro_recentes_selecionado = dados_macro_historicos.iloc[[-1]][melhores_features]

        modelo_final = Ridge(alpha=1.0)
        modelo_final.fit(X_final_selecionado, y_final)
        
        score_r2 = modelo_final.score(X_final_selecionado, y_final)
        print(f"Modelo Ridge treinado com features selecionadas. R²: {score_r2:.4f}")

        # Faz a previsão final 
        previsao_retorno = modelo_final.predict(dados_macro_recentes_selecionado)
        previsoes_q[ativo] = previsao_retorno[0]

    if not previsoes_q:
        print("\nNenhuma previsão pôde ser gerada.")
        return None

    q = pd.Series(previsoes_q).reindex(ativos)

    print("\n--- Vetor Q (Modelo Híbrido) Gerado ---")
    #display((q * 100).map('{:.4f}%'.format))
    
    return q

In [93]:
def coletar_e_imputar_fundamentos(ativos_lista, index_para_df):
    """
    Coleta dados fundamentalistas, preenche valores faltantes com a mediana
    e exibe um relatório detalhado de imputação em formato de tabela.
    """
    # ... (código para coletar os dados e criar df_fund_bruto) ...
    todos_fatores = []
    ativos_processados = []

    print("Coletando dados fundamentalistas brutos para todos os ativos...")
    for ativo in ativos_lista:
        try:
            ticker_sa = ativo if '.SA' in ativo else ativo + '.SA'
            ticker_info = yf.Ticker(ticker_sa).info
            
            fatores = {
                'P_L': ticker_info.get('trailingPE'),
                'ROE': ticker_info.get('returnOnEquity'),
                'P_VP': ticker_info.get('priceToBook'),
                'Div_PL': ticker_info.get('debtToEquity')
            }
            todos_fatores.append(fatores)
            ativos_processados.append(ativo)
        except Exception:
            print(f"AVISO: Falha completa ao buscar dados para {ativo}.")
            todos_fatores.append({'P_L': None, 'ROE': None, 'P_VP': None, 'Div_PL': None})
            ativos_processados.append(ativo)
    
    df_fund_bruto = pd.DataFrame(todos_fatores, index=ativos_processados).replace({None: np.nan})
    medianas = df_fund_bruto.median()
    
    # --- A MUDANÇA ESTÁ AQUI: CONSTRUIR UM DATAFRAME DE RELATÓRIO ---
    print("\n--- Relatório de Processamento de Fatores ---")
    
    # Lista para guardar os dados do relatório
    relatorio_data = []
    
    for ativo in df_fund_bruto.index:
        for fator in df_fund_bruto.columns:
            valor_bruto = df_fund_bruto.loc[ativo, fator]
            
            if pd.isna(valor_bruto):
                status = "Faltante -> IMPUTADO"
                valor_final = medianas.get(fator, 0)
            else:
                status = "Encontrado"
                valor_final = valor_bruto
            
            relatorio_data.append({
                'Ativo': ativo,
                'Fator': fator,
                'Status': status,
                'Valor Final': valor_final
            })

    # Cria o DataFrame do relatório a partir da lista
    df_relatorio = pd.DataFrame(relatorio_data)
    
    # Usa o display (que já está configurado para mostrar tudo)
    #display(df_relatorio)
    # -----------------------------------------------------------------

    # O resto da função continua o mesmo
    df_fund_imputado = df_fund_bruto.fillna(medianas)
    df_fund_imputado.fillna(0, inplace=True)
    
    return df_fund_imputado


def opinioes_RF_com_fundamentos(retornos_historicos, dados_macro_historicos):
    
    # --- CORREÇÃO 1: Usar as variáveis recebidas como parâmetro ---
    print("Padronizando os índices de data...")
    retornos_historicos.index = pd.to_datetime(retornos_historicos.index).to_period('M').to_timestamp('M')
    dados_macro_historicos.index = pd.to_datetime(dados_macro_historicos.index).to_period('M').to_timestamp('M')
    # -----------------------------------------------------------


    # --- ETAPA NOVA: Obter features fundamentalistas já limpas e imputadas ---
    # Coleta os dados de todos os ativos de uma vez
    df_fund_completo = coletar_e_imputar_fundamentos(retornos_historicos.columns, dados_macro_historicos.index)
    # -------------------------------------------------------------------

    previsoes_q = {}
    ativos = retornos_historicos.columns

    for ativo in ativos:
        # Pega as colunas de fatores fundamentalistas relevantes para este ativo
        fatores_fund_ativo = [col for col in df_fund_completo.columns if col.startswith(ativo)]
        df_fatores_fund_ativo = df_fund_completo[fatores_fund_ativo]
        
        # Combina features macro com as fundamentalistas deste ativo
        X_completo = pd.concat([dados_macro_historicos, df_fatores_fund_ativo], axis=1)
        
        # --- CORREÇÃO 2: Garantir que Treino e Previsão usem as mesmas features ---
        
        # Primeiro, define-se o dado para prever (a última linha do conjunto completo de features)
        dados_para_prever = X_completo.iloc[[-1]]
        
        # Agora, prepara-se os dados de TREINO (com lag)
        y = retornos_historicos[ativo]
        X_lagged = X_completo.shift(1)
        dados_completos = pd.concat([y, X_lagged], axis=1).dropna()
        
        y_final = dados_completos[ativo]
        X_final = dados_completos[X_completo.columns] # Features de treino

        if X_final.empty or y_final.empty:
            print(f"ERRO: Após o preparo, não restaram dados para o ativo {ativo}. Pulando.")
            continue

        print(f"\n--- Treinando modelo para o ativo: {ativo} ({len(X_final)} amostras) ---")
        
        # Treinamento do Modelo
        modelo_rf = RandomForestRegressor(n_estimators=200, max_depth=7, max_features='sqrt',
                                           min_samples_leaf=5, random_state=42, oob_score=True)
        modelo_rf.fit(X_final, y_final)
        print(f"Modelo treinado. OOB Score: {modelo_rf.oob_score_:.4f}")
        
        # Geração da Previsão (usando 'dados_para_prever', que tem o formato correto)
        previsao_retorno = modelo_rf.predict(dados_para_prever)
        previsoes_q[ativo] = previsao_retorno[0]
        # ----------------------------------------------------

    if not previsoes_q:
        print("\nNenhuma previsão pôde ser gerada.")
        return None

    q = pd.Series(previsoes_q).reindex(ativos)
    #print("\n--- Vetor Q (RF com Fundamentos) Gerado ---")
    #display((q * 100).map('{:.4f}%'.format))
    
    return q

In [94]:
def opinioes_RF_fundamentalista(retornos_mensais):
    """
    Gera o vetor de visões (Q) usando um modelo Random Forest para cada ativo,
    baseado em features FUNDAMENTALISTAS e de MOMENTUM (corrigido).
    """
    print("--- Gerando Visões com Modelo Fundamentalista/Momentum ---")

    ativos_lista = retornos_mensais.columns.tolist()

    # --- ETAPA 1: Obter features fundamentalistas ---
    # (Esta parte permanece a mesma)
    df_fundamentos = coletar_e_imputar_fundamentos(ativos_lista, retornos_mensais.index)
    
    # --- ETAPA 2: Criar Features de Momentum (Versão Corrigida) ---
    print("\nCalculando features de Momentum específicas para cada ativo...")
    df_momentum = pd.DataFrame(index=retornos_mensais.index)
    
    # Itera sobre as janelas de tempo desejadas
    for n in [3, 6, 12]:
        # Calcula o retorno acumulado para TODOS os ativos de uma vez
        retornos_acumulados = retornos_mensais.rolling(n).sum()
        
        # Renomeia as colunas para incluir o sufixo do momentum (ex: 'ABEV3' -> 'ABEV3_mom_3m')
        retornos_acumulados_renomeado = retornos_acumulados.add_suffix(f'_mom_{n}m')
        
        # Junta as novas colunas de momentum ao nosso DataFrame principal de momentum
        df_momentum = pd.concat([df_momentum, retornos_acumulados_renomeado], axis=1)
    
    print("Features de Momentum criadas com sucesso.")
    # -----------------------------------------------------------------
    
    # --- ETAPA 3: Loop de Treinamento e Previsão ---
    previsoes_q = {}

    for ativo in ativos_lista:
        print(f"\n--- Processando ativo: {ativo} ---")
        
        # Pega as features fundamentalistas deste ativo
        # (agora o df_fundamentos tem ativos no índice)
        features_fund_ativo = df_fundamentos.loc[ativo].to_dict()
        df_fund_ativo = pd.DataFrame([features_fund_ativo], index=retornos_mensais.index)
        
        # Seleciona as colunas de momentum relevantes para este ativo
        colunas_mom_ativo = [col for col in df_momentum.columns if col.startswith(ativo)]
        df_mom_ativo = df_momentum[colunas_mom_ativo]
        
        # Combina as features para criar o X deste ativo
        X = pd.concat([df_fund_ativo, df_mom_ativo], axis=1)
        y = retornos_mensais[ativo]
        
        # Preparação dos dados de treino (com lag)
        X_lagged = X.shift(1)
        dados_completos = pd.concat([y, X_lagged], axis=1).dropna()
        
        y_final = dados_completos[ativo]
        X_final = dados_completos[X.columns]

        if X_final.empty or y_final.empty:
            print(f"ERRO: Não há dados suficientes para o ativo {ativo}. Pulando.")
            previsoes_q[ativo] = 0 # Adiciona 0 como previsão neutra para não quebrar o código
            continue

        print(f"Treinando modelo para {ativo} ({len(X_final)} amostras)...")

        # Treinamento do Modelo Random Forest
        modelo_rf = RandomForestRegressor(
            n_estimators=200, max_depth=7, max_features='sqrt',
            min_samples_leaf=5, random_state=42, oob_score=True
        )
        modelo_rf.fit(X_final, y_final)
        print(f"Modelo treinado. OOB Score: {modelo_rf.oob_score_:.4f}")
        
        # Geração da Previsão
        dados_para_prever = X.iloc[[-1]]
        previsao_retorno = modelo_rf.predict(dados_para_prever)
        previsoes_q[ativo] = previsao_retorno[0]

    q = pd.Series(previsoes_q).reindex(ativos_lista)
    print("\n--- Vetor Q (Modelo Fundamentalista) Gerado ---")
    #display((q * 100).map('{:.4f}%'.format))
    
    return q

In [95]:
def obter_distribuicao_de_visoes_RF(retornos_historicos, dados_macro_historicos):
    """
    Treina um modelo Random Forest para cada ativo e retorna um dicionário
    com a DISTRIBUIÇÃO das previsões de cada árvore individual da floresta.
    """
    distribuicao_visoes = {}
    ativos = retornos_historicos.columns

    # Loop para treinar um modelo para cada ativo
    for ativo in ativos:
        # (Aqui entra todo o seu código de preparação de X_final e y_final)
        # ...
        y = retornos_historicos[ativo]
        X = dados_macro_historicos
        X_lagged = X.shift(1)
        dados_completos = pd.concat([y, X_lagged], axis=1).dropna()
        y_final = dados_completos[ativo]
        X_final = dados_completos[X.columns]
        
        if X_final.empty:
            print(f"AVISO: Sem dados para treinar {ativo}. A visão será neutra (0).")
            distribuicao_visoes[ativo] = np.array([0.0]) # Visão neutra
            continue

        # Treinamento do Modelo Random Forest
        modelo_rf = RandomForestRegressor(n_estimators=200, max_depth=7, max_features='sqrt',
                                           min_samples_leaf=5, random_state=42)
        modelo_rf.fit(X_final, y_final)
        
        # Geração da Previsão
        dados_para_prever = X.iloc[[-1]]
        
        # --- A MÁGICA ESTÁ AQUI ---
        # Pega a previsão de cada árvore individualmente
        previsoes_arvores = [tree.predict(dados_para_prever) for tree in modelo_rf.estimators_]
        
        # Achata a lista de arrays para uma lista simples de números
        distribuicao_visoes[ativo] = np.array(previsoes_arvores).flatten()
        # -------------------------

    return distribuicao_visoes

In [96]:
#pip install google-genai

In [97]:
#pip install openai

In [98]:
# import os
# from openai import OpenAI

# # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings. 
# # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
# client = OpenAI(
#     base_url="https://models.github.ai/inference",
#     api_key=os.environ["GITHUB_TOKEN"],
# )

# response = client.chat.completions.create(
#     messages=[
#         {
#             "role": "system",
#             "content": "",
#         },
#         {
#             "role": "user",
#             "content": "What is the capital of France?",
#         }
#     ],
#     model="openai/gpt-4o",
#     temperature=1,
#     max_tokens=4096,
#     top_p=1
# )

# print(response.choices[0].message.content)


In [99]:
def opinioes_LLM(retornos_mensais, dados_macro_historicos):
    """
    Gera a matriz Q de visões (n x 1) usando a API de Modelos do GitHub.
    """
    
    # Os prompts não mudam. A tarefa para a IA é a mesma.
    system_prompt = """
    You are a quantitative financial analyst AI. Your task is to predict an asset's return based on historical data.
    Analyze the provided data and estimate the average monthly return for the next month.
    
    Output Format:
    Return a single float value that represents the predicted average monthly return for the stock, without any additional commentary or explanation. For example: 0.015 or -0.005.
    """

    user_prompt_template = """
    Please provide the prediction for the following asset:
    - Ticker: {ticker}
    - Monthly Returns Time-Series:
    {retornos_mensais_str}

    - Macroeconomics Data Time-Series:
    {dados_macro_historicos_str}
    """

    client = None
    try:
        # Carregue seu TOKEN DO GITHUB do seu arquivo
        with open('../acessos.txt', 'r') as f:
            # Assumindo que o arquivo agora contém o seu GitHub PAT
            github_token = f.readline().strip()
            #print(github_token)
        
        # 1. (GRANDE MUDANÇA) Inicializar o cliente OpenAI com os dados do GitHub
        client = OpenAI(
            base_url="https://models.github.ai/inference",
            api_key=github_token,
        )
        print("Cliente OpenAI configurado para usar o endpoint do GitHub com sucesso.")

    except Exception as e:
        print(f"Ocorreu um erro ao configurar o cliente: {e}")

    if client is None:
        print("Cliente não inicializado. Retornando matriz de zeros.")
        return np.zeros((n_ativos, 1))

    views = []
    print(f"\nIniciando geração de visões para {n_ativos} ativos com a API do GitHub...")

    global ativos
    ativos = [ticker.replace('.SA', '') for ticker in ativos]

    for ticker in ativos:
        print(f"Processando ativo: {ticker}...")
        try:
            # Esta parte da lógica não muda nada
            stock_returns = retornos_mensais[ticker]
            returns_str = stock_returns.to_string()
            macro_str = dados_macro_historicos.to_string()

            user_prompt = user_prompt_template.format(
                ticker=ticker,
                retornos_mensais_str=returns_str,
                dados_macro_historicos_str=macro_str
            )
            
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
            
            # 2. (PEQUENA MUDANÇA) Chamar a API usando o nome de modelo específico do GitHub
            response = client.chat.completions.create(
                model="openai/gpt-4o",  # Use o nome exato da documentação
                messages=messages,
                temperature=0.2 # Recomendo uma temperatura baixa para maior consistência
            )
            
            # Esta parte não muda, a estrutura da resposta é a mesma
            predicted_return_str = response.choices[0].message.content.strip()
            predicted_return_float = float(predicted_return_str)
            
            views.append(predicted_return_float)
            print(f"  -> Visão para {ticker}: {predicted_return_float:.4f}")

        except Exception as e:
            print(f"  -> ERRO: Ocorreu um problema ao processar {ticker}: {e}. Usando 0.0 como padrão.")
            views.append(0.0)

    q_matrix = np.array(views).reshape(-1, 1)
    return q_matrix

In [100]:
def black_litterman(pi, q, matriz_covarianca_mensal, dados_macro_historicos ):
    # T é o número de observações no histórico disponível NESTE PONTO do backtest
    T = len(dados_macro_historicos)
    print(T)

    tau =  1.0 / T

    omega = np.zeros((n_ativos, n_ativos))

    num_opinioes = n_ativos # Mudar se fizer tbm com visões relativas 

    for i in range(num_opinioes):
        for j in range(num_opinioes):
            if i == j:

                omega[i][j] = np.dot( tau * np.dot(P[i], matriz_covarianca_mensal), P[i].T) #variância das visões do investidor multiplicada pelo escalar tau 

    if not isinstance(matriz_covarianca_mensal, np.ndarray):
        matriz_covarianca_mensal = np.array(matriz_covarianca_mensal)


    qpt = np.dot(matriz_covarianca_mensal, P.T)

    pi = np.array(pi).reshape(-1, 1)          # (n, 1)
    q = np.array(q).reshape(-1, 1)

    retornos = np.array(pi) + np.array(tau * np.dot (qpt,
                        np.dot(np.linalg.inv(np.array(np.dot(P, qpt)) + np.array(omega)),
                        np.array(q)-np.array(np.dot(P,pi)))))

    retornos = retornos.flatten().tolist()

    print("Retornos previstos pelo modelo: ")
    # resultado

    print(n_ativos)
    for i in range(n_ativos):
        print(ativos[i] +': %.2f' %(retornos[i]*100)+'%')
    
    return retornos

In [101]:
def volatilidade(peso, cov_matrix):
    """Calcula a volatilidade da carteira."""
    return np.sqrt(np.dot(peso.T, np.dot(cov_matrix, peso)))

def retorno_esperado(peso, retornos_modelo):
    """Calcula o retorno esperado da carteira."""
    return np.dot(retornos_modelo, peso)

def neg_ratio(peso, retornos_modelo, cov_matrix):
    """
    Calcula o negativo do Sharpe Ratio.
    Esta é a nossa função objetivo para a minimização.
    """
    vol = volatilidade(peso, cov_matrix)
    if vol == 0:
        return np.inf  # Retorna infinito se a volatilidade for zero para evitar divisão por zero

    ret = retorno_esperado(peso, retornos_modelo)
    sharpe = ret / vol
    return -sharpe


limites   = tuple((0.05,1) for x in range(n_ativos))    

restricao = (
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},             
    )     

# Chute inicial: uma carteira com pesos iguais para todos os ativos
chute_inicial = n_ativos * [1. / n_ativos]

def otimizacao(retornos_modelo, matriz_covarianca_mes):
    max_ratio = scpopt.minimize(neg_ratio,
                            x0=chute_inicial,
                            args=(retornos_modelo, matriz_covarianca_mes),  
                            method='SLSQP',
                            bounds=limites,
                            constraints=restricao
                           )
    
    vetor_otimizados = []

    if max_ratio.success:
        print(f"Otimização bem-sucedida: {max_ratio.message}")

        pesos_otimizados = max_ratio.x

        retorno_otimizado = retorno_esperado(pesos_otimizados, retornos_modelo)
        vetor_otimizados.append(retorno_otimizado)

        volatilidade_otimizada = volatilidade(pesos_otimizados,  matriz_covarianca_mes)
        vetor_otimizados.append(volatilidade_otimizada)

        sharpe_otimizado = retorno_otimizado / volatilidade_otimizada
        vetor_otimizados.append(sharpe_otimizado)

        print(f"\nRetorno Esperado da Carteira Otimizada: {retorno_otimizado*100:.2f}% (mensal)")
        print(f"Volatilidade da Carteira Otimizada: {volatilidade_otimizada*100:.2f}% (mensal)")
        print(f"Índice Sharpe Otimizado: {sharpe_otimizado:.4f}")

        print("\nPesos da Carteira Otimizada:")
        pesos_df = pd.DataFrame({'Ativo': ativos, 'Peso (%)': pesos_otimizados * 100})
        pesos_df.set_index('Ativo', inplace=True)
        display(pesos_df)

    else:
        print(f"A otimização falhou: {max_ratio.message}")
    
    return vetor_otimizados

def otimizar_pesos(retornos_modelo, matriz_cov):
    """
    Executa a otimização e retorna APENAS o vetor de pesos da carteira.
    Ideal para ser usada dentro de um loop de backtest.
    """
    n_ativos = len(retornos_modelo)
    limites = tuple((0.05, 1) for _ in range(n_ativos)) 
    restricao = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    chute_inicial = n_ativos * [1. / n_ativos]

    max_ratio = scpopt.minimize(neg_ratio,
                                x0=chute_inicial,
                                args=(retornos_modelo, matriz_cov),
                                method='SLSQP',
                                bounds=limites,
                                constraints=restricao)
    
    if max_ratio.success:
        return max_ratio.x  # <-- RETORNA APENAS OS PESOS
    else:
        return None


In [102]:
w_mkt = capitalizacao()

retornos_mes = retornos_mensais(data_inicio, data_fim )
retornos_mes.index = pd.to_datetime(retornos_mes.index).to_period('M').to_timestamp('M')

volatilidade_mes = volatilidade_mensal(retornos_mes)
matriz_covarianca_mes = matriz_covarianca_mensal(retornos_mes)



Buscando a taxa de câmbio USD/BRL...
Taxa de câmbio atual: R$ 5.34

Buscando dados de capitalização/AUM para cada ativo...


Processando Ativos: 100%|██████████| 10/10 [00:04<00:00,  2.49it/s]


--- Capitalização de Mercado / AUM (em BRL) - COMPLETO ---


ABEV3    192,720,650,240
BBAS3    122,787,479,552
EGIE3     33,689,657,344
ITUB4    389,550,374,912
LREN3     16,132,463,616
MRFG3     17,631,934,464
PETR4    424,012,808,192
SLCE3      7,266,680,832
VALE3    247,589,175,296
WEGE3    155,702,280,192
Name: Capitalizacao_BRL, dtype: object


--- Pesos da Carteira de Mercado (w_mkt)


ABEV3    11.99%
BBAS3     7.64%
EGIE3     2.10%
ITUB4    24.24%
LREN3     1.00%
MRFG3     1.10%
PETR4    26.38%
SLCE3     0.45%
VALE3    15.41%
WEGE3     9.69%
Name: Capitalizacao_BRL, dtype: object

[*********************100%***********************]  10 of 10 completed



Soma dos pesos: 1.0000
               Selic
Date                
2012-01-01  0.008917
2012-02-01  0.008667
2012-03-01  0.008183
2012-04-01  0.007792
2012-05-01  0.007392
...              ...
2025-03-01  0.011308
2025-04-01  0.011792
2025-05-01  0.012125
2025-06-01  0.012283
2025-07-01  0.012417

[163 rows x 1 columns]
Selic baixada e convertida para taxa mensal.
Calculando os retornos excedentes (descontando a taxa livre de risco)...

--- Amostra dos Retornos Excedentes Calculados ---


,ABEV3,BBAS3,EGIE3,ITUB4,LREN3,MRFG3,PETR4,SLCE3,VALE3,WEGE3
Date,,,,,,,,,,
2012-02-29,0.102599,0.008110,-0.007349,0.038413,0.094012,0.265521,-0.019305,-0.014937,-0.034730,-0.028366
2012-03-31,0.071965,-0.066015,0.066019,-0.059994,-0.050038,0.078038,-0.048474,0.195399,-0.023221,0.035163
2012-04-30,0.073974,-0.092739,-0.003827,-0.159532,-0.028355,-0.099716,-0.095029,-0.041711,-0.014809,0.012551
2012-05-31,-0.081580,-0.172059,0.022753,-0.030572,-0.035609,-0.149540,-0.114372,0.023959,-0.107652,0.024854
2012-06-30,0.002074,-0.027933,0.097920,-0.041571,-0.038436,0.019040,-0.054084,0.055160,0.058395,-0.072043


In [103]:
def obter_pesos_resampled(retornos_historicos, n_simulacoes=100):
    """
    Versão garantida que retorna um dicionário.
    """
    n_ativos = retornos_historicos.shape[1]
    lista_pesos_otimos = []
    
    # (O código do loop de simulação continua o mesmo)
    for _ in range(n_simulacoes):
        retornos_simulados = retornos_historicos.sample(n=len(retornos_historicos), replace=True)
        mu_simulado = retornos_simulados.mean()
        sigma_simulado = retornos_simulados.cov()
        chute_inicial = n_ativos * [1. / n_ativos]
        limites = tuple((0.05, 1.0) for _ in range(n_ativos))
        restricao = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        res = scpopt.minimize(neg_ratio, x0=chute_inicial, args=(mu_simulado, sigma_simulado),
                              method='SLSQP', bounds=limites, constraints=restricao)
        if res.success:
            lista_pesos_otimos.append(res.x)

    if not lista_pesos_otimos:
        return None

    pesos_resampled = np.mean(lista_pesos_otimos, axis=0)
    pesos_resampled /= pesos_resampled.sum()
    
    # Calculando as métricas para retornar no dicionário
    retornos_esperados_para_analise = retornos_historicos.mean()
    matriz_cov_para_analise = retornos_historicos.cov()
    
    # Monta o dicionário
    resultados = {
        "pesos": pesos_resampled,
        "retornos_bl": retornos_esperados_para_analise, # Reutilizando a chave para consistência
        "matriz_cov": matriz_cov_para_analise
    }
    
    print("Dentro da função, prestes a retornar um dicionário.")
    return resultados

# # --- TESTANDO A FUNÇÃO ---
# print("Chamando a função corrigida...")
# # Supondo que 'retornos_mes' é o seu DataFrame histórico completo
# resultados_teste = obter_pesos_resampled_CORRIGIDA(retornos_mes)

# if resultados_teste is not None:
#     print(f"\nTipo do objeto retornado: {type(resultados_teste)}")
    
#     # Agora, o teste final: tentar acessar a chave 'pesos'
#     try:
#         pesos_teste = resultados_teste['pesos']
#         print(f"\n✅ SUCESSO! A chave 'pesos' foi acessada.")
#         print("Vetor de pesos extraído:")
#         print(pesos_teste)
#     except (TypeError, KeyError) as e:
#         print(f"\n❌ FALHA! Ainda não foi possível acessar a chave 'pesos'. Erro: {e}")
# else:
#     print("\nA função retornou None (falhou).")

In [104]:
def obter_pesos_otimizados(retornos_historicos, dados_macro_historicos, ml):

    matriz_covarianca_mes = matriz_covarianca_mensal(retornos_historicos)

    delta_calculado = delta(retornos_historicos , w_mkt)

    vetor_pi = pi(delta_calculado, w_mkt, matriz_covarianca_mes)

    #retornos_reais_do_mes = dados_completos.loc[data, retornos_mes.columns]
    
    #if ml == -1: 
        #resultados = obter_pesos_resampled(retornos_historicos)
    if ml == -2:
        matriz_cov = retornos_historicos.cov()
        # O input de retorno é a média histórica
        retornos_esperados = retornos_historicos.mean()
        
        # Otimiza usando a média histórica como 'retornos_bl'
        pesos_otimizados = otimizar_pesos(retornos_esperados, matriz_cov)
        
        if pesos_otimizados is not None:
            # Retorna o dicionário no mesmo formato das outras estratégias
            resultados = {
                "pesos": pesos_otimizados,
                "retornos_bl": retornos_esperados, # Guardamos a média histórica aqui
                "matriz_cov": matriz_cov
            }
            return resultados
        else:
            return None

    #retorno_realizado = np.dot(pesos_otimizados, retornos_reais_do_mes) não precisa pq esses modelos não prevem retornos


    if ml == 6:
        print("\n--- INICIANDO MÉTODO DE BOOTSTRAP DAS VISÕES DE ML ---")
        
        # 1. Obtenha a distribuição das visões uma única vez
        # Usaremos o modelo Híbrido como exemplo para gerar a distribuição
        dist_visoes = obter_distribuicao_de_visoes_RF(retornos_historicos, dados_macro_historicos)
        if dist_visoes is None:
            return None

        # 2. Defina o número de simulações
        n_simulacoes = 100
        lista_de_pesos_otimizados = []
        lista_de_q_simulados = [] # Para calcular o q médio no final

        print(f"Iniciando {n_simulacoes} simulações de bootstrap...")
        for _ in range(n_simulacoes):
            # a. Crie um vetor q simulado
            q_simulado = {}
            for ativo, distribuicao in dist_visoes.items():
                q_simulado[ativo] = np.random.choice(distribuicao)
            q_vetor_simulado = pd.Series(q_simulado).reindex(retornos_historicos.columns)
            lista_de_q_simulados.append(q_vetor_simulado)

            # b. Rode o Black-Litterman e a otimização com este q simulado
            retornos_bl_simulado = black_litterman(vetor_pi, q_vetor_simulado, matriz_covarianca_mes, dados_macro_historicos)
            pesos_simulados = otimizar_pesos(retornos_bl_simulado, matriz_covarianca_mes)
            
            if pesos_simulados is not None:
                lista_de_pesos_otimizados.append(pesos_simulados)

        # 3. Calcule a carteira final
        if not lista_de_pesos_otimizados:
            print("Nenhuma otimização foi bem-sucedida durante a simulação.")
            return None
        
        # Pega a média dos pesos de todas as simulações
        pesos_otimizados = np.mean(lista_de_pesos_otimizados, axis=0)
        pesos_otimizados /= pesos_otimizados.sum() # Normaliza para garantir soma 1

        # Para consistência do relatório, calculamos um 'retornos_bl' final
        # usando a média de todos os 'q' simulados
        q_medio = pd.concat(lista_de_q_simulados, axis=1).mean(axis=1)
        retornos_bl = black_litterman(vetor_pi, q_medio, matriz_covarianca_mes, dados_macro_historicos)

        # Retorna o dicionário no formato esperado
        resultados = {
            "pesos": pesos_otimizados,
            "retornos_bl": retornos_bl,
            "matriz_cov": matriz_covarianca_mes
        }
        return resultados

    # --- Lógica Padrão (Otimização Única para ml = 1 a 6) ---
    else:

        if ml == 1:
            q = opinioes_RF(retornos_historicos, dados_macro_historicos)
        elif ml == 2:
            q = opinioes_LR(retornos_historicos, dados_macro_historicos)
        elif ml == 3:
            q = opinioes_HIBRIDO(retornos_historicos, dados_macro_historicos)

        elif ml == 4:
            q = opinioes_RF_com_fundamentos(retornos_historicos, dados_macro_historicos)
        elif ml == 5: 
            q = opinioes_RF_fundamentalista(retornos_historicos)

        elif ml == 7:
            q = opinioes_LLM(retornos_historicos, dados_macro_historicos) 

        retornos_bl = black_litterman(vetor_pi, q, matriz_covarianca_mes, dados_macro_historicos )

        pesos_otimizados = otimizar_pesos(retornos_bl, matriz_covarianca_mes)

        if pesos_otimizados is not None:

            resultados = {
                "pesos": pesos_otimizados,
                "retornos_bl": retornos_bl,
                "matriz_cov": matriz_covarianca_mes
            }
            return resultados
        else:
            # Se a otimização falhar, retorne None
            return None

Comparativo dos retornos esperados

In [105]:
# # Célula de análise e benchmark (versão final e correta)

# def comparativo_ibov(resultados, pesos_otimizados, retornos_bl, matriz_cov ):

#     if resultados is not None:


#         # Para sua Carteira
#         # O 'retornos_bl' já é um vetor de retornos excedentes (se você seguiu a correção anterior)
#         # Se não, precisa subtrair a selic média aqui também. Vamos assumir que ele já é excedente.
#         retorno_carteira_excedente = retorno_esperado(pesos_otimizados, retornos_bl)
#         volat_carteira = volatilidade(pesos_otimizados, matriz_cov)
#         sharpe_carteira_correto = retorno_carteira_excedente / volat_carteira

#         # 4. Montar e exibir a tabela
#         benchmark_df = pd.DataFrame({
#             'Métrica': ['Retorno Excedente Mensal (%)', 'Volatilidade Mensal (%)', 'Índice Sharpe'],
#             'Carteira Otimizada BL': [retorno_carteira_excedente * 100, volat_carteira * 100, sharpe_carteira_correto],
#             'Benchmark Ibovespa': [ibov_retorno_excedente_medio * 100, ibov_volatilidade * 100, ibov_sharpe_correto]
#         })

#         benchmark_df.set_index('Métrica', inplace=True)

#         # 4. Exibir a tabela formatada
#         print("\n--- Tabela de Benchmark Comparativa (Formatada) ---")
#         display(benchmark_df.style.format({
#             'Carteira Otimizada BL': '{:.4f}',
#             'Benchmark Ibovespa': '{:.4f}'
#         }).set_caption("Comparativo de Desempenho Ex-Ante")
#         .background_gradient(cmap='viridis', subset=['Carteira Otimizada BL', 'Benchmark Ibovespa']))

#         #return ibov_retornos_excedentes
#         #return resultados_ibov

Backtesting

In [106]:
from contextlib import redirect_stdout, redirect_stderr
# Unir todos os seus dados em um único lugar para facilitar o fatiamento

def backtesting(retornos_mes, ml):

    dados_completos = retornos_mes.join(dados_macro_combinados)

    # Definir o período de backtest
    datas_backtest = dados_completos.index[dados_completos.index >= data_backtest]

    # Lista para guardar os resultados
    retornos_da_estrategia = []
    datas_dos_retornos = []

    for data in tqdm(datas_backtest, desc="Executando Backtest",  disable=True):
        # 1. Fatiar os dados: tudo o que aconteceu ATÉ a data atual
        data_treino = dados_completos.loc[:data].iloc[:-1] # Pega tudo ATÉ o mês anterior
        if data_treino.empty: continue
        
        retornos_treino = data_treino[retornos_mes.columns]
        macro_treino = data_treino[dados_macro_combinados.columns]
        
        # 2. Rodar a estratégia para obter os pesos para o próx mês
        with open(os.devnull, 'w') as fnull:
            with redirect_stdout(fnull), redirect_stderr(fnull):
                if (ml == -1):
                    resultados_otimizacao = obter_pesos_resampled(retornos_treino)
                else:
                    resultados_otimizacao = obter_pesos_otimizados(retornos_treino, macro_treino, ml) 
        
        # Verifica se a otimização funcionou e retornou o dicionário
        if resultados_otimizacao is None:
            print(f"Não foi possível otimizar para a data {data}. Pulando.")
            continue
            
        pesos = resultados_otimizacao['pesos']
            
        #Calcular o retorno realizado no mês seguinte
        # Pegamos os retornos que realmente aconteceram na data 'data'
        retornos_reais_do_mes = dados_completos.loc[data, retornos_mes.columns]
        retorno_realizado_portfolio = np.dot(pesos, retornos_reais_do_mes)
        
        # Guardar resultado
        retornos_da_estrategia.append(retorno_realizado_portfolio)
        datas_dos_retornos.append(data)

    # Criar a série de retornos da sua estratégia
    estrategia_retornos_realizados = pd.Series(retornos_da_estrategia, index=datas_dos_retornos)

    return estrategia_retornos_realizados

Comparação real com a ibov

In [107]:
# Análise da sua estratégia
def analise( estrategia_retornos_realizados, i):
    media_log_retornos_estrategia = estrategia_retornos_realizados.mean()

    estrategia_retorno_medio = np.exp(media_log_retornos_estrategia) - 1

    estrategia_volatilidade = estrategia_retornos_realizados.std()
    estrategia_sharpe = estrategia_retorno_medio / estrategia_volatilidade

    print("\nPadronizando o índice do Ibovespa para fim de mês para o benchmark...")
    # Garante que o índice do Ibovespa também use o fim do mês, como o da sua estratégia
    ibov_retornos_excedentes.index = pd.to_datetime(ibov_retornos_excedentes.index).to_period('M').to_timestamp('M')
    print("Índice do Ibovespa padronizado. ")

    # Fatiar os retornos do IBOV para o mesmo período do backtest
    ibov_retornos_backtest = ibov_retornos_excedentes.loc[estrategia_retornos_realizados.index]
    media_log_retorno_medio_backtest = ibov_retornos_backtest.mean()

    ibov_retorno_medio_backtest = np.exp(media_log_retorno_medio_backtest) - 1
    
    ibov_volatilidade_backtest = ibov_retornos_backtest.std()
    ibov_sharpe_backtest = ibov_retorno_medio_backtest / ibov_volatilidade_backtest

    print(f"Média dos Log-Retornos:       {media_log_retornos_estrategia:.8f}")
    print(f"Média Geométrica (Composta): {estrategia_retorno_medio:.8f}")
    print(f"Diferença Absoluta:          {abs(media_log_retornos_estrategia - estrategia_retorno_medio):.8f}")

    metodo = ""
    if (i == 1):
        metodo = "RF"
    elif (i == 2):
        metodo = "LR"
    elif (i == 3):
        metodo = "Hibrido"
    elif (i == 4):
        metodo = "RF com macro e análise fundamentalista"
    elif(i == 5):
        metodo = "RF apenas com análise fundamentalista"
    elif (i == 7):
        metodo = "LLM"
    elif (i == 6):
        metodo = "RF + Resampled Effiency"
    elif (i == -1):
        metodo = "Resampled Effiency"
    
    coluna_nome = f'Sua Estratégia BL {metodo}'

    # Tabela de comparação final
    benchmark_final_df = pd.DataFrame({
        'Métrica': ['Retorno Realizado Mensal', 'Volatilidade Mensal', 'Índice Sharpe'],
        coluna_nome : [
            estrategia_retorno_medio.item() * 100, 
            estrategia_volatilidade.item() * 100, 
            estrategia_sharpe.item()
        ],
        'Benchmark Ibovespa': [
            ibov_retorno_medio_backtest.item() * 100, 
            ibov_volatilidade_backtest.item() * 100, 
            ibov_sharpe_backtest.item()
        ]
    })
    benchmark_final_df.set_index('Métrica', inplace=True)

    # --- Exibição Estilizada Simples ---
    print("\n--- Tabela de Benchmark Final (Backtest Realizado) ---")

    display(benchmark_final_df.style.format({
            # Formata ambas as colunas com 4 casas decimais para consistência
            coluna_nome : '{:.4f}',
            'Benchmark Ibovespa': '{:.4f}'
        }).set_caption("<b>Comparativo de Desempenho Realizado (Backtest)</b>") # Legenda corrigida
        .background_gradient(cmap='viridis', subset=[coluna_nome , 'Benchmark Ibovespa']))


Código Principal

In [108]:
retornos_mes = retornos_mensais(data_inicio, data_fim )
retornos_mes.index = pd.to_datetime(retornos_mes.index).to_period('M').to_timestamp('M')

volatilidade_mes = volatilidade_mensal(retornos_mes)
matriz_covarianca_mes = matriz_covarianca_mensal(retornos_mes)

resultados_backtest = {}
metodos_a_testar = {
    -2: "Markowitz Clássico",
    -1: "Eficiência Reamostrada",
     1: "BL com RF",
     2: "BL com LR",
     3: "BL Híbrido"
     # Adicione outros métodos que queira testar
}

#tirei llm para teste, muito demorado
for i in range(1, 4): # i = 1 : Só RF, i = 2 : Só LR, i = 3 : RF + LR,  i = 4 : RF + fundamentalista com macro , i = 5 : RF e fundamentalista, i = 6 : LLM (Chatgpt)

    resultados = obter_pesos_otimizados(retornos_mes, dados_macro_combinados, i)

    if resultados is not None:
        
        pesos_otimizados = resultados["pesos"]
        print("\nPesos dos ativos após otimização")
        for j in range(n_ativos):
            print(ativos[j] +': %.2f' %(pesos_otimizados[j]*100)+'%')

        retornos_bl = resultados["retornos_bl"]
        matriz_cov = resultados["matriz_cov"]
        print("Resultados da otimização extraídos com sucesso.")

        #resultados_ibov = comparativo_ibov(resultados, pesos_otimizados, retornos_bl, matriz_cov)
        retornos_estrategia = backtesting(retornos_mes, i)

        resultados_backtest[i] = retornos_estrategia
        
        analise( retornos_estrategia, i)

        

    else:
        print("Otimização falhou. Não é possível prosseguir com a análise.")

[*********************100%***********************]  10 of 10 completed


               Selic
Date                
2012-01-01  0.008917
2012-02-01  0.008667
2012-03-01  0.008183
2012-04-01  0.007792
2012-05-01  0.007392
...              ...
2025-03-01  0.011308
2025-04-01  0.011792
2025-05-01  0.012125
2025-06-01  0.012283
2025-07-01  0.012417

[163 rows x 1 columns]
Selic baixada e convertida para taxa mensal.
Calculando os retornos excedentes (descontando a taxa livre de risco)...

--- Amostra dos Retornos Excedentes Calculados ---


,ABEV3,BBAS3,EGIE3,ITUB4,LREN3,MRFG3,PETR4,SLCE3,VALE3,WEGE3
Date,,,,,,,,,,
2012-02-29,0.102599,0.008110,-0.007349,0.038413,0.094012,0.265521,-0.019305,-0.014937,-0.034730,-0.028366
2012-03-31,0.071965,-0.066015,0.066019,-0.059994,-0.050038,0.078038,-0.048474,0.195399,-0.023221,0.035163
2012-04-30,0.073974,-0.092739,-0.003827,-0.159532,-0.028355,-0.099716,-0.095029,-0.041711,-0.014809,0.012551
2012-05-31,-0.081580,-0.172059,0.022753,-0.030572,-0.035609,-0.149540,-0.114372,0.023959,-0.107652,0.024854
2012-06-30,0.002074,-0.027933,0.097920,-0.041571,-0.038436,0.019040,-0.054084,0.055160,0.058395,-0.072043


Amostra dos retornos históricos do portfólio de mercado:

Prêmio de Risco Mensal Médio (Numerador): 0.1326%
Variância Mensal do Mercado (Denominador): 0.005061
O coeficiente de aversão ao risco (δ) calculado é: 0.2620
Padronizando os índices de data...

--- Treinando modelo para o ativo: ABEV3 (158 amostras) ---
Modelo treinado. OOB Score: 0.0245

--- Treinando modelo para o ativo: BBAS3 (158 amostras) ---
Modelo treinado. OOB Score: -0.0131

--- Treinando modelo para o ativo: EGIE3 (158 amostras) ---
Modelo treinado. OOB Score: -0.0519

--- Treinando modelo para o ativo: ITUB4 (158 amostras) ---
Modelo treinado. OOB Score: -0.0308

--- Treinando modelo para o ativo: LREN3 (158 amostras) ---
Modelo treinado. OOB Score: -0.0929

--- Treinando modelo para o ativo: MRFG3 (158 amostras) ---
Modelo treinado. OOB Score: -0.0477

--- Treinando modelo para o ativo: PETR4 (158 amostras) ---
Modelo treinado. OOB Score: 0.0486

--- Treinando modelo para o ativo: SLCE3 (158 amostras) ---
Modelo tr

,Sua Estratégia BL RF,Benchmark Ibovespa
Métrica,,
Retorno Realizado Mensal,0.2149,-0.0301
Volatilidade Mensal,4.0292,4.3551
Índice Sharpe,0.0533,-0.0069


Amostra dos retornos históricos do portfólio de mercado:

Prêmio de Risco Mensal Médio (Numerador): 0.1326%
Variância Mensal do Mercado (Denominador): 0.005061
O coeficiente de aversão ao risco (δ) calculado é: 0.2620
Modelo para ABEV3 treinado. R² (ajuste de treino): 0.0773
Modelo para BBAS3 treinado. R² (ajuste de treino): 0.0963
Modelo para EGIE3 treinado. R² (ajuste de treino): 0.0714
Modelo para ITUB4 treinado. R² (ajuste de treino): 0.0684
Modelo para LREN3 treinado. R² (ajuste de treino): 0.0933
Modelo para MRFG3 treinado. R² (ajuste de treino): 0.0686
Modelo para PETR4 treinado. R² (ajuste de treino): 0.1286
Modelo para SLCE3 treinado. R² (ajuste de treino): 0.0494
Modelo para VALE3 treinado. R² (ajuste de treino): 0.0319
Modelo para WEGE3 treinado. R² (ajuste de treino): 0.1431

--- Vetor Q (Regressão Linear) Gerado ---
160
Retornos previstos pelo modelo: 
10
VALE3.SA: 0.06%
PETR4.SA: 0.19%
ITUB4.SA: 0.06%
MRFG3.SA: 0.14%
ABEV3.SA: 0.13%
WEGE3.SA: 0.07%
EGIE3.SA: 0.23%
LREN3.S

,Sua Estratégia BL LR,Benchmark Ibovespa
Métrica,,
Retorno Realizado Mensal,0.0456,-0.0301
Volatilidade Mensal,4.1372,4.3551
Índice Sharpe,0.0110,-0.0069


Amostra dos retornos históricos do portfólio de mercado:

Prêmio de Risco Mensal Médio (Numerador): 0.1326%
Variância Mensal do Mercado (Denominador): 0.005061
O coeficiente de aversão ao risco (δ) calculado é: 0.2620
--- Gerando Visões com Modelo HÍBRIDO (RF para Seleção, Ridge para Previsão) ---
Selecionando as 2 features mais importantes para cada ativo.

Ativo: ABEV3 | Melhores Features: ['Investimento Direto no País_diff', 'PETROLEO_BRENT_diff']
Modelo Ridge treinado com features selecionadas. R²: 0.0498

Ativo: BBAS3 | Melhores Features: ['PETROLEO_BRENT_diff', 'CAMBIO_USDBRL_diff']
Modelo Ridge treinado com features selecionadas. R²: 0.0113

Ativo: EGIE3 | Melhores Features: ['DOW_JONES_diff', 'PETROLEO_BRENT_diff']
Modelo Ridge treinado com features selecionadas. R²: 0.0084

Ativo: ITUB4 | Melhores Features: ['PETROLEO_BRENT_diff', 'CAMBIO_USDBRL_diff']
Modelo Ridge treinado com features selecionadas. R²: 0.0029

Ativo: LREN3 | Melhores Features: ['VIX_diff', 'DOW_JONES_diff']


,Sua Estratégia BL Hibrido,Benchmark Ibovespa
Métrica,,
Retorno Realizado Mensal,0.3371,-0.0301
Volatilidade Mensal,3.9375,4.3551
Índice Sharpe,0.0856,-0.0069


In [109]:
# def resampled_efficiency():

#     i = -1 # Código para resampled
#     resultados =  obter_pesos_resampled(retornos_mes)
#     print(resultados)
#     #print("\nPesos dos ativos após otimização")
#     #for j in range(n_ativos):
#        #print(ativos[j] +': %.2f' %(pesos_otimizados[j]*100)+'%')

#     pesos_otimizados = resultados["pesos"]
#     retornos = resultados["retornos_resamp"] # Deixar bl só para não mudar a função anterior
#     matriz = resultados["matriz_cov"]
#     print("Resultados da otimização extraídos com sucesso.")

#     resultados_ibov = comparativo_ibov(resultados, pesos_otimizados, retornos, matriz)
#     retornos_estrategia = backtesting(retornos_mes, i)

#     analise(resultados_ibov , retornos_estrategia, i)

# resampled_efficiency()


In [110]:
def analisar_estrategia_resampled():
    """
    Função completa para analisar a estratégia de Eficiência Reamostrada.
    Ela calcula as métricas esperadas (ex-ante) e roda o backtest
    para obter as métricas realizadas (ex-post).
    """
    print("--- Iniciando Análise Completa da Estratégia Resampled Efficiency ---")

    # Supondo que 'retornos_mes' é seu DataFrame com o histórico completo (2012-2024)
    # e está disponível no escopo do seu notebook.
    retornos_historicos = retornos_mes

    # --- 1. ANÁLISE EX-ANTE (A "PROMESSA" DO MODELO) ---

    print("\nCalculando as métricas esperadas (Ex-Ante)...")
    
    # a. Calcule os inputs para a análise
    # A "melhor estimativa" de retorno é a média histórica
    retornos_esperados_para_analise = retornos_historicos.mean()
    matriz_cov_para_analise = retornos_historicos.cov()

    # b. Obtenha os pesos da estratégia Resampled usando o histórico completo
    resultados_resampled = obter_pesos_resampled(retornos_historicos)
    pesos_otimizados_resampled = resultados_resampled["pesos"]
    
    #print(pesos_otimizados_resampled)
    
    if resultados_resampled is None:
        print("Falha ao obter os pesos da estratégia Resampled. Análise abortada.")
        return

    # c. Calcule as métricas esperadas da carteira
    # Multiplicamos os pesos (resultado da estratégia) pelos retornos esperados (média histórica)
    retorno_carteira_esperado = retorno_esperado(pesos_otimizados_resampled, retornos_esperados_para_analise)
    volat_carteira_esperada = volatilidade(pesos_otimizados_resampled, matriz_cov_para_analise)
    sharpe_carteira_esperado = retorno_carteira_esperado / volat_carteira_esperada

    print("Métricas esperadas calculadas com sucesso.")


    # --- 2. ANÁLISE EX-POST (O BACKTEST) ---
    
    print("\nIniciando o backtest da estratégia Resampled...")
    # Usamos o código -1 para o Resampled, como você sugeriu
    retornos_estrategia_realizados = backtesting(retornos_historicos, -1) 

    resultados_backtest[-1] = retornos_estrategia_realizados 
    
    # Métricas realizadas a partir do resultado do backtest
    retorno_realizado_medio = retornos_estrategia_realizados.mean()
    volat_realizada = retornos_estrategia_realizados.std()
    sharpe_realizado = retorno_realizado_medio / volat_realizada
    print("Backtest concluído.")

    # --- 3. EXIBIÇÃO DOS RESULTADOS ---

    # Tabela comparativa final
    # (Adicionando o Ibovespa para um benchmark completo)
    # ibov_retornos_backtest = ibov_retornos.loc[retornos_estrategia_realizados.index]
    # ibov_retorno_medio = ibov_retornos_backtest.mean().item()
    # ibov_volatilidade = ibov_retornos_backtest.std().item()
    # ibov_sharpe = ibov_retorno_medio / ibov_volatilidade

    tabela_final = pd.DataFrame({
        'Métrica': ['Retorno Mensal (%)', 'Volatilidade Mensal (%)', 'Índice Sharpe'],
        'Promessa (Ex-Ante)': [retorno_carteira_esperado * 100, volat_carteira_esperada * 100, sharpe_carteira_esperado],
        'Realidade (Backtest)': [retorno_realizado_medio * 100, volat_realizada * 100, sharpe_realizado]
        #'Benchmark Ibovespa': [ibov_retorno_medio * 100, ibov_volatilidade * 100, ibov_sharpe]
    })
    tabela_final.set_index('Métrica', inplace=True)
    
    display(tabela_final.style.format('{:.4f}').set_caption("<b>Análise Completa: Resampled Efficiency</b>"))

# --- Para executar a sua análise ---
# Lembre-se de ter as funções auxiliares (retorno_esperado, volatilidade, etc.) definidas
# e a variável 'retornos_mes' carregada.
analisar_estrategia_resampled()

--- Iniciando Análise Completa da Estratégia Resampled Efficiency ---

Calculando as métricas esperadas (Ex-Ante)...


/home/caio/Documentos/opt_cod/.venv/lib/python3.10/site-packages/scipy/optimize/_slsqp_py.py:435: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds

/home/caio/Documentos/opt_cod/.venv/lib/python3.10/site-packages/scipy/optimize/_slsqp_py.py:439: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds

/home/caio/Documentos/opt_cod/.venv/lib/python3.10/site-packages/scipy/optimize/_slsqp_py.py:493: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



Dentro da função, prestes a retornar um dicionário.
Métricas esperadas calculadas com sucesso.

Iniciando o backtest da estratégia Resampled...
Backtest concluído.


,Promessa (Ex-Ante),Realidade (Backtest)
Métrica,,
Retorno Mensal (%),0.4542,-0.1359
Volatilidade Mensal (%),5.1521,4.5253
Índice Sharpe,0.0882,-0.0300


In [111]:
def analisar_estrategia_markowitz():
    """
    Função completa para analisar a estratégia de Markowitz Clássico.
    Calcula as métricas esperadas (ex-ante) e roda o backtest (ex-post).
    """
    print("--- Iniciando Análise Completa da Estratégia Markowitz Clássico ---")

    # Supondo que 'retornos_mes' é o DataFrame com o histórico completo
    retornos_historicos = retornos_mes

    # --- 1. ANÁLISE EX-ANTE (A "PROMESSA" DO MODELO) ---
    print("\nCalculando as métricas esperadas (Ex-Ante)...")
    
    # Inputs para a análise: média e covariância de todo o período
    retornos_esperados_para_analise = retornos_historicos.mean()
    matriz_cov_para_analise = retornos_historicos.cov()

    # Obter os pesos da estratégia Markowitz usando o histórico completo
    pesos_otimizados_markowitz = obter_pesos_otimizados(retornos_historicos, dados_macro_combinados, -2 )['pesos']
    
    if pesos_otimizados_markowitz is None:
        print("Falha ao obter os pesos da estratégia Markowitz. Análise abortada.")
        return

    # Calcular as métricas esperadas da carteira
    retorno_carteira_esperado = retorno_esperado(pesos_otimizados_markowitz, retornos_esperados_para_analise)
    volat_carteira_esperada = volatilidade(pesos_otimizados_markowitz, matriz_cov_para_analise)
    sharpe_carteira_esperado = retorno_carteira_esperado / volat_carteira_esperada

    print("Métricas esperadas calculadas com sucesso.")

    # --- 2. ANÁLISE EX-POST (O BACKTEST) ---
    print("\nIniciando o backtest da estratégia Markowitz...")
    # Usamos o código -2 para Markowitz
    retornos_estrategia_realizados = backtesting(retornos_historicos, -2)


    resultados_backtest[-2] = retornos_estrategia_realizados 

    retorno_realizado_medio = retornos_estrategia_realizados.mean()
    volat_realizada = retornos_estrategia_realizados.std()
    sharpe_realizado = retorno_realizado_medio / volat_realizada
    
    tabela_final = pd.DataFrame({
        'Métrica': ['Retorno Mensal (%)', 'Volatilidade Mensal (%)', 'Índice Sharpe'],
        'Promessa (Ex-Ante)': [retorno_carteira_esperado * 100, volat_carteira_esperada * 100, sharpe_carteira_esperado],
        'Realidade (Backtest)': [retorno_realizado_medio * 100, volat_realizada * 100, sharpe_realizado]
        #'Benchmark Ibovespa': [ibov_retorno_medio * 100, ibov_volatilidade * 100, ibov_sharpe]
    })
    tabela_final.set_index('Métrica', inplace=True)
    
    display(tabela_final.style.format('{:.4f}').set_caption("<b>Análise Completa: Modelo Markowitz</b>"))


analisar_estrategia_markowitz()

--- Iniciando Análise Completa da Estratégia Markowitz Clássico ---

Calculando as métricas esperadas (Ex-Ante)...
Amostra dos retornos históricos do portfólio de mercado:

Prêmio de Risco Mensal Médio (Numerador): 0.1326%
Variância Mensal do Mercado (Denominador): 0.005061
O coeficiente de aversão ao risco (δ) calculado é: 0.2620
Métricas esperadas calculadas com sucesso.

Iniciando o backtest da estratégia Markowitz...


,Promessa (Ex-Ante),Realidade (Backtest)
Métrica,,
Retorno Mensal (%),0.5720,-0.0091
Volatilidade Mensal (%),5.3457,4.9211
Índice Sharpe,0.1070,-0.0019


In [112]:
#pip install nbformat

In [113]:
# --- 3. ANÁLISE DE DESEMPENHO AO LONGO DO TEMPO ---

# Converte o dicionário de resultados em um único DataFrame
df_retornos = pd.DataFrame(resultados_backtest)

# Fatia o benchmark para o mesmo período dos resultados do backtest
# (Usamos o índice do primeiro resultado como referência)
primeira_estrategia = list(resultados_backtest.keys())[0]
indice_backtest = resultados_backtest[primeira_estrategia].index
ibov_retornos_backtest = ibov_retornos_excedentes.loc[indice_backtest]

# Adiciona o Ibovespa ao DataFrame para comparação completa
df_retornos['Ibovespa'] = ibov_retornos_backtest

# Agora o 'df_retornos' está exatamente no formato que você precisa
# para a análise de curva de patrimônio.

# --- Gráfico de Retorno Acumulado (Curva de Patrimônio) ---
print("\nGerando gráfico de retorno acumulado...")

# Calcula o crescimento de R$ 1,00 (retorno acumulado)
# Lembre-se que estes são retornos EXCEDENTES. Para o patrimônio real, 
# teríamos que somar a Selic de volta, mas para comparar as estratégias, isso é o ideal.
df_acumulado = (1 + df_retornos).cumprod()

# Plotando com Plotly
import plotly.graph_objects as go
fig = go.Figure()
for col in df_acumulado.columns:
    fig.add_trace(go.Scatter(x=df_acumulado.index, y=df_acumulado[col], mode='lines', name=col))

fig.update_layout(title_text='<b>Prêmio de risco acumulado: Comparativo de Estratégias</b>',
                  yaxis_title='Prêmio de risco acumulado',
                  xaxis_title='Data',
                  legend_title='Estratégia')
fig.show()


Gerando gráfico de retorno acumulado...


In [114]:
# Supondo que 'df_retornos' (com os retornos excedentes das estratégias),
# 'selic_mensal', 'ibov_retornos_brutos' (brutos) e 'indice_backtest' já existem.

# Fatia a Selic para o período do backtest
selic_fatiada = selic_mensal.loc[indice_backtest]

# Adiciona a Selic de volta aos retornos excedentes para obter os retornos brutos das estratégias
df_retornos_brutos = df_retornos.add(selic_fatiada['Taxa_Livre_Risco'], axis=0)

# Fatia os retornos brutos do Ibovespa para o mesmo período
ibov_retornos_brutos_fatiado = ibov_retornos_brutos.loc[indice_backtest]


# --- A CORREÇÃO ESTÁ AQUI ---
# Adicionamos a série de retornos do Ibovespa como uma nova coluna
# ao DataFrame que será usado para o gráfico.
df_retornos_brutos['Ibovespa'] = ibov_retornos_brutos_fatiado
# ---------------------------


print("\n--- Gráfico 1: Curva de Patrimônio Real (Baseado nos Retornos Brutos) ---")
df_acumulado_bruto = (1 + df_retornos_brutos).cumprod()

# O resto do seu código de plotagem agora funcionará perfeitamente
# e incluirá a linha do Ibovespa no loop.
fig_bruto = go.Figure()
for col in df_acumulado_bruto.columns:
    fig_bruto.add_trace(go.Scatter(x=df_acumulado_bruto.index, y=df_acumulado_bruto[col], mode='lines', name=col))
    
fig_bruto.update_layout(title_text='<b>Crescimento de R$ 1,00 (Patrimônio Real)</b>',
                            yaxis_title='Patrimônio Acumulado (R$)', legend_title='Estratégia')
fig_bruto.show()


--- Gráfico 1: Curva de Patrimônio Real (Baseado nos Retornos Brutos) ---


In [115]:
# --- 1. CÁLCULO DO SHARPE MÓVEL ---

# Define a janela de cálculo. 12 meses (1 ano) é um padrão comum.
janela_movel = 12

# Calcula a média móvel dos retornos
retornos_moveis = df_retornos.rolling(window=janela_movel).mean()

# Calcula o desvio padrão móvel (volatilidade) dos retornos
volatilidade_movel = df_retornos.rolling(window=janela_movel).std()

# Calcula o Sharpe Ratio Móvel e o anualiza para melhor interpretação
# (Multiplicamos pela raiz quadrada de 12 para anualizar)
sharpe_movel_anualizado = (retornos_moveis / volatilidade_movel) * np.sqrt(12)

# Remove os NaNs iniciais (os 11 primeiros meses não têm dados suficientes)
sharpe_movel_anualizado = sharpe_movel_anualizado.dropna()


# --- 2. PLOTAGEM DO GRÁFICO ---

print("\n--- Gráfico de Índice Sharpe Móvel (Janela de 12 Meses) ---")

fig = go.Figure()

# Adiciona uma linha para cada estratégia no DataFrame
for estrategia in sharpe_movel_anualizado.columns:
    fig.add_trace(go.Scatter(
        x=sharpe_movel_anualizado.index,    # Eixo X = Datas
        y=sharpe_movel_anualizado[estrategia], # Eixo Y = Valores do Sharpe
        mode='lines',
        name=estrategia
    ))

# Adiciona uma linha horizontal em y=0 para referência
fig.add_hline(y=0, line_width=1, line_dash="dash", line_color="grey")

fig.update_layout(
    title_text="<b>Índice Sharpe Móvel Anualizado (Janela de 12 Meses)</b>",
    yaxis_title="Índice Sharpe Anualizado",
    xaxis_title="Data",
    legend_title="Estratégia"
)

fig.show()


--- Gráfico de Índice Sharpe Móvel (Janela de 12 Meses) ---


In [116]:
#pip install statsmodels


In [117]:
import statsmodels.api as sm

# Supondo que 'resultados_backtest' e 'ibov_retornos_excedentes' já existem
# e que 'metodos_a_testar' é o dicionário com os nomes das estratégias

# É melhor iterar pelas chaves do dicionário de resultados,
# para garantir que só analisamos as estratégias que rodaram com sucesso.
for i in resultados_backtest.keys():
    print(f"\n--- Análise de Regressão para a Estratégia: {metodos_a_testar.get(i, i)} ---")
    
    y = resultados_backtest[i]
    
    # --- A CORREÇÃO ESTÁ AQUI ---
    # Alinhamos o X (Ibovespa) para ter exatamente o mesmo índice (datas) que o y (estratégia)
    X = ibov_retornos_excedentes.loc[y.index]
    # ---------------------------

    # Adiciona a constante (para o alfa) ao X já alinhado
    X = sm.add_constant(X)
    
    # Agora a regressão funcionará, pois os índices de y e X são idênticos
    modelo = sm.OLS(y, X).fit()

    print(modelo.summary()) # Exibe a tabela completa da regressão

    # Extrai os parâmetros
    alfa_mensal = modelo.params['const']
    beta = modelo.params.get(ibov_retornos_excedentes.columns[0], modelo.params.iloc[1]) # Forma robusta de pegar o beta
    
    print(f"\nAlfa Mensal (habilidade do gestor): {alfa_mensal*100:.4f}%")
    print(f"Beta (exposição ao mercado): {beta:.4f}")
    
    # Interpretação do Alfa
    p_valor_alfa = modelo.pvalues['const']
    if p_valor_alfa < 0.05:
        print("-> O Alfa é estatisticamente significante (p < 0.05), indicando que a estratégia gerou um retorno que não é explicado pelo mercado. ✅")
    else:
        print("-> O Alfa não é estatisticamente significante (p >= 0.05), indicando que o desempenho pode ser atribuído ao acaso. ❌")


--- Análise de Regressão para a Estratégia: BL com RF ---
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.759
Model:                            OLS   Adj. R-squared:                  0.752
Method:                 Least Squares   F-statistic:                     106.9
Date:                Mon, 22 Sep 2025   Prob (F-statistic):           4.92e-12
Time:                        18:33:10   Log-Likelihood:                 90.637
No. Observations:                  36   AIC:                            -177.3
Df Residuals:                      34   BIC:                            -174.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------